## Adding libraries.

In [19]:
options(warn = -1)
library(tidyverse)
library(elo)

## Import dataset.

In [29]:
ff_scores <- read.csv(file = "C:/Users/chris/Documents/Datasets/FFL/ff_scores.csv")
ff_scores <- ff_scores %>%
  select(date, team_1, team_2, win) %>%
  arrange(date)

## Generate teams table with initial elo value = 1500.

In [77]:
teams <- data.frame(team = unique(ff_scores$team_1)) %>%
  mutate(elo = 1500)

## Calculate changes in elo rating based on weekly results.

In [79]:
team1_elo_pre_vector <- c()
team2_elo_pre_vector <- c()
team1_elo_post_vector <- c()
team2_elo_post_vector <- c()

for (i in seq_len(nrow(ff_scores))) {
  match <- ff_scores[i, ]
  
  # Pre-match ratings.
  team_1_elo <- subset(teams, team == match$team_1)$elo
  team_2_elo <- subset(teams, team == match$team_2)$elo
  
  # Elo calculation.
  new_elo <- elo.calc(wins.A = match$win,
                      elo.A = team_1_elo,
                      elo.B = team_2_elo,
                      k = 30)
    
  # Post-match ratings  .
  team_1_new_elo <- new_elo[1, 1]
  team_2_new_elo <- new_elo[1, 2]

  team1_elo_pre_vector <- append(team1_elo_pre_vector, team_1_elo)
  team2_elo_pre_vector <- append(team2_elo_pre_vector, team_2_elo)
  team1_elo_post_vector <- append(team1_elo_post_vector, team_1_new_elo)
  team2_elo_post_vector <- append(team2_elo_post_vector, team_2_new_elo)

  # Update team elo ratings table.
  teams <- teams %>%
    mutate(elo = case_when(
        (team == match$team_1) ~ team_1_new_elo,
        (team == match$team_2) ~ team_2_new_elo,
        ((team != match$team_1) & (team != match$team_2)) ~ elo
    ))
}

## Create columns to save pre/post-week elo ratings.

In [80]:
ff_scores$team1_elo_pre <- team1_elo_pre_vector
ff_scores$team2_elo_pre <- team2_elo_pre_vector
ff_scores$team1_elo_post <- team1_elo_post_vector
ff_scores$team2_elo_post <- team2_elo_post_vector

## Elo rating history table.

In [85]:
head(ff_scores)

,date,team_1,team_2,win,team1_elo_pre,team2_elo_pre,team1_elo_post,team2_elo_post
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,2014-01,CHRIS,CARLEN,0,1500,1500,1485.000,1515.000
2,2014-01,ANDREW,KYLE,0,1500,1500,1485.000,1515.000
3,2014-01,STEVE,DREW,1,1500,1500,1515.000,1485.000
4,2014-01,MATT,NATHAN,1,1500,1500,1515.000,1485.000
5,2014-02,MATT,CHRIS,1,1515,1485,1528.708,1471.292
6,2014-02,STEVE,ANDREW,0,1515,1485,1498.708,1501.292


## Current elo rating table.

In [84]:
teams %>%
  arrange(-elo)

team,elo
<chr>,<dbl>
CHRIS,1651.796
ANDREW,1579.214
NATHAN,1535.348
STEVE,1527.988
SAM,1518.152
NICK,1511.848
KYLE,1498.459
CARLEN,1475.735
DREW,1474.429
